In [1]:
pip install pandas plotly

In [2]:
import pandas as pd

df0 = pd.read_csv('data/0_preprocessed.csv')
df0

,timestamp,duration,song,artist,album,song_artist
0,7/5/2019 3:00,2.45,7 rings,Ariana Grande,"thank u, next",7 rings - Ariana Grande
1,7/5/2019 3:01,1.29,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
2,7/5/2019 3:05,3.67,I Don't Care,Ed Sheeran,I Don't Care,I Don't Care - Ed Sheeran
3,7/5/2019 3:08,3.30,Beautiful People,Ed Sheeran,Beautiful People,Beautiful People - Ed Sheeran
4,7/5/2019 3:12,3.32,All Falls Down,Alan Walker,All Falls Down,All Falls Down - Alan Walker
...,...,...,...,...,...,...
47672,9/4/2024 13:36,3.15,The Lazy Song,Bruno Mars,Doo-Wops & Hooligans,The Lazy Song - Bruno Mars
47673,9/4/2024 14:35,2.64,Part Of Me,Katy Perry,Teenage Dream: The Complete Confection,Part Of Me - Katy Perry
47674,9/5/2024 13:50,0.00,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan
47675,9/5/2024 16:08,0.07,Beautiful In White,Shane Filan,Love Always,Beautiful In White - Shane Filan


In [3]:
df0['timestamp'] = pd.to_datetime(df0['timestamp'])

In [4]:
from datetime import datetime, timedelta

start_date = datetime(2019, 7, 1)
end_date = datetime(2024, 10, 1)

timestamps = []
current_date = start_date
while current_date <= end_date:
  timestamps.append(current_date)
  current_date = (current_date + timedelta(days=32)).replace(day=1)

len(timestamps), timestamps[0], timestamps[-1]

(64, datetime.datetime(2019, 7, 1, 0, 0), datetime.datetime(2024, 10, 1, 0, 0))

In [38]:
data_gantt = {
  'timestamp_start': [],
  'timestamp_end': [],
  'most_played_artist': [],
  'most_played_song': [],
  'artist_duration': [],
  'song_duration': []
}

# Iterate over the timestamps
for i in range(len(timestamps) - 1):
  # Get the current timestamp and the next timestamp
  current_timestamp = timestamps[i]
  next_timestamp = timestamps[i+1]

  # Filter the DataFrame based on the timestamps
  filtered_df = df0[(df0['timestamp'] >= current_timestamp) & (df0['timestamp'] < next_timestamp)]

  # Calculate the total duration for each artist
  artist_duration = filtered_df.groupby('artist')['duration'].sum().astype(int)

  # Find the artist with the most played time
  most_played_artist = artist_duration.idxmax() if not artist_duration.empty else None
  most_played_artist_duration = artist_duration.max() if not artist_duration.empty else 0

  # Calculate the total duration for each song
  song_duration = filtered_df.groupby('song')['duration'].sum().astype(int)

  # Find the song with the most played time
  most_played_song = song_duration.idxmax() if not song_duration.empty else None
  most_played_song_duration = song_duration.max() if not song_duration.empty else 0

  # Append the data to the list
  data_gantt['timestamp_start'].append(current_timestamp)
  data_gantt['timestamp_end'].append(next_timestamp)
  data_gantt['most_played_artist'].append(most_played_artist)
  data_gantt['artist_duration'].append(most_played_artist_duration)
  data_gantt['most_played_song'].append(most_played_song)
  data_gantt['song_duration'].append(most_played_song_duration)


# Create a new DataFrame from the data
df_gantt = pd.DataFrame(data_gantt)

# Print the new DataFrame
df_gantt

,timestamp_start,timestamp_end,most_played_artist,most_played_song,artist_duration,song_duration
0,2019-07-01,2019-08-01,Taylor Swift,Everything Has Changed,1041,515
1,2019-08-01,2019-09-01,Taylor Swift,Everything Has Changed,602,192
2,2019-09-01,2019-10-01,Taylor Swift,Perfect,239,84
3,2019-10-01,2019-11-01,Ed Sheeran,a thousand years,166,85
4,2019-11-01,2019-12-01,Taylor Swift,One Call Away,1032,164
...,...,...,...,...,...,...
58,2024-05-01,2024-06-01,Andiez,Suýt Nữa Thì...,2080,1294
59,2024-06-01,2024-07-01,Sơn Tùng M-TP,Nơi Này Có Anh,3358,867
60,2024-07-01,2024-08-01,Sơn Tùng M-TP,Nơi Này Có Anh,851,238
61,2024-08-01,2024-09-01,Sơn Tùng M-TP,Nơi Này Có Anh,2489,706


In [39]:
n_artists_otm = len(df_gantt['most_played_artist'].unique())
n_songs_otm = len(df_gantt['most_played_song'].unique())

In [40]:
df_gantt['most_played_artist_original'] = df_gantt['most_played_artist']
df_gantt['most_played_song_original'] = df_gantt['most_played_song']

In [41]:
if n_artists_otm > 9:
  top_9_artists = df_gantt['most_played_artist'].value_counts().head(9).index
  df_gantt['most_played_artist'] = df_gantt['most_played_artist'].apply(lambda x: f'{n_artists_otm-9} Others' if x not in top_9_artists else x)

In [42]:
if n_songs_otm > 9:
  top_9_songs = df_gantt['most_played_song'].value_counts().head(9).index
  df_gantt['most_played_song'] = df_gantt['most_played_song'].apply(lambda x: f'{n_songs_otm-9} Others' if x not in top_9_songs else x)

In [43]:
df_gantt

,timestamp_start,timestamp_end,most_played_artist,most_played_song,artist_duration,song_duration,most_played_artist_original,most_played_song_original
0,2019-07-01,2019-08-01,Taylor Swift,Everything Has Changed,1041,515,Taylor Swift,Everything Has Changed
1,2019-08-01,2019-09-01,Taylor Swift,Everything Has Changed,602,192,Taylor Swift,Everything Has Changed
2,2019-09-01,2019-10-01,Taylor Swift,13 Others,239,84,Taylor Swift,Perfect
3,2019-10-01,2019-11-01,Ed Sheeran,13 Others,166,85,Ed Sheeran,a thousand years
4,2019-11-01,2019-12-01,Taylor Swift,13 Others,1032,164,Taylor Swift,One Call Away
...,...,...,...,...,...,...,...,...
58,2024-05-01,2024-06-01,Andiez,13 Others,2080,1294,Andiez,Suýt Nữa Thì...
59,2024-06-01,2024-07-01,Sơn Tùng M-TP,Nơi Này Có Anh,3358,867,Sơn Tùng M-TP,Nơi Này Có Anh
60,2024-07-01,2024-08-01,Sơn Tùng M-TP,Nơi Này Có Anh,851,238,Sơn Tùng M-TP,Nơi Này Có Anh
61,2024-08-01,2024-09-01,Sơn Tùng M-TP,Nơi Này Có Anh,2489,706,Sơn Tùng M-TP,Nơi Này Có Anh


In [44]:
gantt_top_artists = df_gantt['most_played_artist'].value_counts()
gantt_top_artists

,count
most_played_artist,
Taylor Swift,36
Ludovico Einaudi,17
Sơn Tùng M-TP,4
Ed Sheeran,1
Frédéric Chopin,1
Joe Hisaishi,1
Lana Del Rey,1
JustaTee,1
Andiez,1


In [45]:
artists_ranking = df_gantt['most_played_artist'].value_counts().to_dict()
artists_y0 = {artist: rank*0.5 - 0.2 for rank, artist in enumerate(artists_ranking)}
artists_y1 = {artist: rank*0.5 + 0.2 for rank, artist in enumerate(artists_ranking)}
for artist in artists_y0:
  print(artist, artists_y0[artist], artists_y1[artist])

Taylor Swift -0.2 0.2
Ludovico Einaudi 0.3 0.7
Sơn Tùng M-TP 0.8 1.2
Ed Sheeran 1.3 1.7
Frédéric Chopin 1.8 2.2
Joe Hisaishi 2.3 2.7
Lana Del Rey 2.8 3.2
JustaTee 3.3 3.7
Andiez 3.8 4.2


In [46]:
gantt_top_songs = df_gantt['most_played_song'].value_counts()
gantt_top_songs

,count
most_played_song,
Nuvole Bianche,23
13 Others,13
Enchanted,11
Sparks Fly,5
Nơi Này Có Anh,5
Everything Has Changed,2
Mặt Trời Của Em,1
Speechless,1
Heal the World,1


In [47]:
top_artists = []
n_top_artists = df_gantt['most_played_artist'].nunique()
i = 0
artists_y = {}
for artist in df_gantt['most_played_artist']:
  if artist not in artists_y:
    artists_y[artist] = n_top_artists / 2 - i
    top_artists.append(artist)
    i += 0.5
print(top_artists)

['Taylor Swift', 'Ed Sheeran', 'Ludovico Einaudi', 'Frédéric Chopin', 'Joe Hisaishi', 'Lana Del Rey', 'JustaTee', 'Andiez', 'Sơn Tùng M-TP']


In [48]:
top_songs = []
n_top_songs = df_gantt['most_played_song'].nunique()
i = 0
songs_y = {}
for song in df_gantt['most_played_song']:
  if song not in songs_y:
    songs_y[song] = n_top_songs / 2 - i
    top_songs.append(song)
    i += 0.5
print(top_songs[::-1])

['Nơi Này Có Anh', 'Born To Die', 'Heal the World', 'Speechless', 'Mặt Trời Của Em', 'Enchanted', 'Nuvole Bianche', 'Sparks Fly', '13 Others', 'Everything Has Changed']


In [49]:
df_gantt['song_y'] = df_gantt['most_played_song'].map(songs_y)
df_gantt['song_y0'] = df_gantt['song_y'] - 0.2
df_gantt['song_y1'] = df_gantt['song_y'] + 0.2
df_gantt['artist_y'] = df_gantt['most_played_artist'].map(artists_y)
df_gantt['artist_y0'] = df_gantt['artist_y'] - 0.2
df_gantt['artist_y1'] = df_gantt['artist_y'] + 0.2

In [50]:
import plotly.figure_factory as ff
import plotly.express as px

In [51]:
colors = px.colors.qualitative.Bold
colors_songs = {song : color for song, color in zip(df_gantt['most_played_song'].value_counts().index, colors)}
colors_artists = {artist : color for artist, color in zip(df_gantt['most_played_artist'].value_counts().index, colors)}

In [52]:
df_gantt['song_color'] = df_gantt['most_played_song'].map(colors_songs)
df_gantt['artist_color'] = df_gantt['most_played_artist'].map(colors_artists)

In [53]:
gantt_songs, gantt_artists = [], []
for i, row in df_gantt.iterrows():
    song = row['most_played_song']
    artist = row['most_played_artist']
    start = row['timestamp_start']
    finish = row['timestamp_end']
    n_weeks_song = gantt_top_songs[song]
    n_weeks_artist = gantt_top_artists[artist]
    task_song = song + f' {n_weeks_song}m'
    task_artist = artist + f' {n_weeks_artist}m'

    gantt_songs.append(
        dict(
            Task=task_song,
            Start=start,
            Finish=finish,
            Resource=song
        )
    )

    gantt_artists.append(
        dict(
            Task=task_artist,
            Start=start,
            Finish=finish,
            Resource=artist
        )
    )

In [54]:
fig_songs = ff.create_gantt(gantt_songs, title='Most-played Song of the Month',colors=colors_songs, index_col='Resource', group_tasks=True, height=500, bar_width=0.5)
fig_songs.show()

In [55]:
fig_artists = ff.create_gantt(gantt_artists, title='Most-played Artist of the Month',colors=colors_artists, index_col='Resource', group_tasks=True, height=500, bar_width=0.5)
fig_artists.show()

In [56]:
df_gantt.to_csv('3_gantt_top_otm.csv', index=False)

In [57]:
print(['Taylor Swift', 'Ed Sheeran', 'Ludovico Einaudi', 'Frédéric Chopin', 'Joe Hisaishi', 'Lana Del Rey', 'JustaTee', 'Andiez', 'Sơn Tùng M-TP'][::-1])

['Sơn Tùng M-TP', 'Andiez', 'JustaTee', 'Lana Del Rey', 'Joe Hisaishi', 'Frédéric Chopin', 'Ludovico Einaudi', 'Ed Sheeran', 'Taylor Swift']
